Created: 2020.08.13

Modified: 2020.08.22

### Prepare TEST csv files for inference (pts. 5.xx)

Get csv **test set** files (e.g. ixi_test_mk_3d.csv) with paths to **3D NIFTI images**, convert each 3D image path to a set of **2D slice** paths and save to resulting csv file in the **2.4_train_val_2d_path_tables** folder.

Those paths to each 2D slice are needed during prediction process to concatenate to a part of training data (for efficiency reason) and feed with a learner.


In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import glob
import pandas as pd
from pathlib import Path

In [3]:
import sys
sys.path.append('functions')

%aimport functions00, functions01
from functions00 import *
from functions01 import *

import functions24 as fun24
import functions50 as fun50

In [4]:
mk_get_host_info()

**********************************************************************************************************************************
Settings:
	HOST:  mmiv-ml-titan
	PATH_ROOT_DATA:  /data-10tb/shared/skull/train-3d-iso
	PATH_GIT_HUB:  /data-10tb/marek/github_codes/skull-stripping-1/fastai
	PATH_2D: /data-10tb/shared/skull

3D DFs paths (_mk_3D):
	IXI_TEST_3D: /data-10tb/marek/github_codes/skull-stripping-1/fastai/2.2_train_valid_test_sets/ixi_test_mk_3d.csv
	TEST_3D : /data-10tb/marek/github_codes/skull-stripping-1/fastai/2.2_train_valid_test_sets/test_mk_3d.csv
	TRAIN_VAL_3D: /data-10tb/marek/github_codes/skull-stripping-1/fastai/2.2_train_valid_test_sets/train_val_mk_3d.csv
	NFBS_TEST_3D: /data-10tb/marek/github_codes/skull-stripping-1/fastai/2.2_train_valid_test_sets/nfbs_test_mk_3d.csv

Error files (pickle):
	ERROR_FILES: /data-10tb/marek/github_codes/skull-stripping-1/fastai/2.2_train_valid_test_sets/error_files
**********************************************************************

In [5]:
len(mk_get_error_file_list())

26

#### Global parameters

In [8]:
test1_name = 'TEST_3D'
test2_name = 'IXI_TEST_3D'
test3_name = 'NFBS_TEST_3D'

test1 = TEST_3D
test2 = IXI_TEST_3D
test3 = NFBS_TEST_3D

############################
TEST_SET = test1
TEST_SET_NAME = test1_name
#############################

SAVE = 1

#### Load TEST df

In [9]:
save = 1

for PLANE in ['axial-2d', 'coronal-2d', 'sagittal-2d']:
    # load TEST df
    df1 = pd.read_csv(TEST_SET)
    df1['usage_txt'] = TEST_SET_NAME.lower()
    df1['usage'] = True
    
    # remove  failed images based on Sathiesh's error_list file
    mk_remove_error_flies_from_df(df1, mk_get_error_file_list(), column_name='t1_pth_dgx')
    mk_remove_error_flies_from_df(df1, mk_get_error_file_list(), column_name='bias_pth_dgx')

    # Drop some columns
    cols = ['t1_pth_titan','bias_pth_titan', 'mask_nn_pth_titan','usage', 'usage_txt' ]
    df = df1[cols]

    # Lists o files to process
    #usg = df.usage[0]
    #usg_txt = df.usage_txt[0]

    # Get lists of all 3D NIFTI images: to t1, bias, mask
    t1_list = df.t1_pth_titan
    bias_list = df.bias_pth_titan
    mask_list = df.mask_nn_pth_titan
    #print(len(t1_list), len(bias_list), len(mask_list))

    
    usg = df.usage[0]
    usg_txt = df.usage_txt[0]

    df_t1_list = []
    df_bias_list = []

    for t,b,m in zip(t1_list, bias_list, mask_list):

        pngs_t1 = fun50.get_pngs_for_mask(t, PLANE, 'test')
        pngs_bias = fun50.get_pngs_for_mask(b, PLANE, 'test')
        pngs_msk = fun50.get_pngs_for_mask(m, PLANE, 'test')

        # df for t1_mask-testset
        df_t1 = fun24.create_pngs_df_for_mask(pngs_t1, pngs_msk, usg, usg_txt, t1=True)
        # df for bias_mask-testset
        df_bias = fun24.create_pngs_df_for_mask(pngs_bias, pngs_msk, usg, usg_txt, t1=False)

        df_t1_list.append(df_t1)
        df_bias_list.append(df_bias)

    # why such a way to create new df???? I'dont remember ;)
    df_t1_all = pd.concat(df_t1_list)
    df_bias_all = pd.concat(df_bias_list)
    
    # Create column name for newly created df's: t1_path for t1, and bias_path for bias
    col_t1 = ['root', 't1_path','image_full_path', 'mask_full_path', 'usage_txt', 'usage']
    col_bias = ['root', 'bias_path','image_full_path', 'mask_full_path', 'usage_txt', 'usage']

    # Update column name in newly created df's
    df_t1_all = df_t1_all[col_t1]
    df_bias_all = df_bias_all[col_bias]

    # Save names: e.g.: IXI_TEST_3D_t1-mask-axial-2d
    t1_save_name = f'{TEST_SET_NAME}-t1-mask-{PLANE}.csv'
    bias_save_name = f'{TEST_SET_NAME}-bias-mask-{PLANE}.csv'
    
    # save folder name
    save_folder_name = '2.4_train_val_2d_path_tables'

    if SAVE:
        fun24.save_df(df_t1_all, PLANE, t1_save_name, save_folder_name)
        fun24.save_df(df_bias_all, PLANE, bias_save_name, save_folder_name)
    print(f'\n******************************************************** {PLANE}...done :D *********************************************************\n')

 *** column_name=t1_pth_dgx ***
	None of the files from the error_list is contained in a df!
	New df shape: (934, 17)

 *** column_name=bias_pth_dgx ***
	Removed follong files from the df (7 files): 
		ADNI_027_S_0118_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20090227154801572_S62836_I137255.anat_T1_biascorr_iso.nii.gz
		sub-031358_T1w.anat_T1_biascorr_iso.nii.gz
		sub-031584_T1w.anat_T1_biascorr_iso.nii.gz
		sub-25743_ses-1_T1w.anat_T1_biascorr_iso.nii.gz
		sub-25645_ses-1_T1w.anat_T1_biascorr_iso.nii.gz
		sub-30967_ses-1_T1w.anat_T1_biascorr_iso.nii.gz
		sub-30772_ses-3_T1w.anat_T1_biascorr_iso.nii.gz
	New df shape: (927, 17)

Saved files:
	/data-10tb/marek/github_codes/skull-stripping-1/fastai/2.4_train_val_2d_path_tables/TEST_3D-t1-mask-axial-2d.csv
	/data-10tb/shared/skull/axial-2d/TEST_3D-t1-mask-axial-2d.csv
Saved files:
	/data-10tb/marek/github_codes/skull-stripping-1/fastai/2.4_train_val_2d_path_tables/TEST_3D-bias-mask-axial-2d.csv
	/data-10tb/shared/skull/axial-2d/TEST_